In [ ]:
pip install arviz

     |████████████████████████████████| 1.6MB 3.8MB/s 
     |████████████████████████████████| 4.7MB 27.8MB/s 
     |████████████████████████████████| 768kB 35.9MB/s 
     |████████████████████████████████| 317kB 34.1MB/s 
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1


In [ ]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import arviz
import pickle
import cmdstanpy
import json
import pandas as pd

In [ ]:
with open('/content/drive/MyDrive/TFM-Crowd learning/multinomial.json') as json_file:
    data = json.load(json_file)

In [ ]:
dades={'Tasca': data['t_A'], 'Worker': data['w_A'], 'Anotació': data['ann'] }
df = pd.DataFrame(data=dades)
df

,Tasca,Worker,Anotació
0,0,14,2
1,0,13,2
2,0,4,2
3,0,2,2
4,0,7,1
...,...,...,...
9995,999,17,2
9996,999,2,2
9997,999,13,2
9998,999,14,2


In [ ]:
sm_ext = pystan.StanModel(file='/content/drive/MyDrive/TFM-Crowd learning/Fitxers STAN/extern_classes.stan')


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5e4939feb462bdcc96651d9a4509501e NOW.


In [ ]:
w = len(np.unique(df['Worker']))
t = len(np.unique(df['Tasca']))
a = len(df['Anotació'])
k = len(np.unique(df['Anotació']))
num_classes_externes = 1
k2=k+num_classes_externes
t_A = np.array(df['Tasca'])+1
w_A = np.array(df['Worker'])+1
ann = np.array(df['Anotació'])+1
tau_prior = np.array([1,1,1])
pi_prior = np.ones((k,k2))

dades_sm_ext = {'w':w,'t':t,'a':a,'k':k,'k2':k2,'t_A':t_A,'w_A':w_A,'ann':ann,'tau_prior':tau_prior,'pi_prior':pi_prior} 

In [ ]:
inicial = np.array([[0.91,0.03,0.03,0.03],
                    [0.03,0.91,0.03,0.03],
                    [0.03,0.03,0.91,0.03]])

In [ ]:
pi_ini = np.zeros((w,k,k2))
for yw in range(w):
  pi_ini[yw] = inicial
dades_inicials_gen = {'pi':pi_ini}

In [ ]:
optim = sm_ext.optimizing(data=dades_sm_ext,init=dades_inicials_gen)
optim

OrderedDict([('tau', array([0.11252257, 0.29708451, 0.59039292])),
             ('pi',
              array([[[8.50347658e-01, 7.91487509e-02, 7.05035912e-02, 8.36414271e-14],
                      [2.19097745e-01, 5.88714383e-01, 1.92187750e-01, 1.22642694e-07],
                      [1.47658250e-01, 9.12999941e-02, 7.61041746e-01, 9.31813093e-09]],
              
                     [[6.54470920e-01, 1.26597066e-01, 2.18931987e-01, 2.75059208e-08],
                      [3.07690740e-01, 5.17229727e-01, 1.75079396e-01, 1.36806753e-07],
                      [1.30578244e-01, 1.10735663e-01, 7.58686081e-01, 1.23414184e-08]],
              
                     [[6.24035079e-01, 8.48555505e-02, 2.91109112e-01, 2.58758061e-07],
                      [2.13787591e-01, 6.02157076e-01, 1.84055198e-01, 1.34893658e-07],
                      [7.40944991e-02, 1.08489483e-01, 8.17416001e-01, 1.70793957e-08]],
              
                     [[6.92333222e-01, 9.32334797e-02, 2.14433281e-01, 1.

In [ ]:
extern_indexs = np.random.randint(0,1000,size=150)
extern_indexs

array([ 56, 712, 707, 395, 684, 638, 492, 492, 849, 803, 670, 325, 659,
       746, 864, 789, 737, 276, 942, 240, 313, 738, 595, 632,  81, 178,
       428, 133, 329, 830, 692, 574, 586,  64, 990, 361, 363, 683,  83,
       113,  77, 840,  76,  35, 708, 236, 428, 995, 838, 503, 668, 330,
        47, 369, 283, 759, 607, 415, 979, 164, 525, 491,  41, 887, 671,
       111, 725,  31, 774, 187, 277, 141, 553, 909, 178, 757, 411, 349,
       466, 607, 753, 602, 577, 827, 710, 227, 170,   6, 736, 585, 604,
       573, 229, 610, 606, 346, 550, 982, 993, 423, 403, 999, 800,  51,
       413, 355, 678, 754, 513, 202, 585, 818, 748, 857, 585, 768, 350,
        40, 727, 798, 803, 550, 956, 241, 914, 979, 404,  31, 492, 906,
       375, 837, 243,  69, 781,  35, 880, 942, 705, 128, 793, 270, 808,
       938, 325, 306, 943, 836, 133, 948])

In [ ]:
df_modified = df.copy()
anotations = df_modified['Anotació'].values
for index in extern_indexs:
  anotations[index] = 3
 

In [ ]:
df_modified = df_modified.drop(columns= ['Anotació'])
df_modified['Anotació'] = anotations
df_modified

,Tasca,Worker,Anotació
0,0,14,2
1,0,13,2
2,0,4,2
3,0,2,2
4,0,7,1
...,...,...,...
9995,999,17,2
9996,999,2,2
9997,999,13,2
9998,999,14,2


In [ ]:
w = len(np.unique(df_modified['Worker']))
t = len(np.unique(df_modified['Tasca']))
a = len(df_modified['Anotació'])
k = 3
num_classes_externes = 1
k2=k+num_classes_externes
t_A = np.array(df_modified['Tasca'])+1
w_A = np.array(df_modified['Worker'])+1
ann = np.array(df_modified['Anotació'])+1
tau_prior = np.array([1,1,1])
pi_prior = np.ones((k,k2))

dades_sm_ext_modified = {'w':w,'t':t,'a':a,'k':k,'k2':k2,'t_A':t_A,'w_A':w_A,'ann':ann,'tau_prior':tau_prior,'pi_prior':pi_prior} 

In [ ]:
inicial = np.array([[0.91,0.03,0.03,0.03],
                    [0.03,0.91,0.03,0.03],
                    [0.03,0.03,0.91,0.03]])

In [ ]:
pi_ini = np.zeros((w,k,k2))
for yw in range(w):
  pi_ini[yw] = inicial
dades_inicials_gen = {'pi':pi_ini}

In [ ]:
optim = sm_ext.optimizing(data=dades_sm_ext_modified ,init=dades_inicials_gen)
optim

OrderedDict([('tau', array([0.1138177 , 0.29464779, 0.59153451])),
             ('pi',
              array([[[8.38713312e-01, 7.72456667e-02, 6.97248673e-02, 1.43161543e-02],
                      [2.13998630e-01, 5.90605437e-01, 1.95394249e-01, 1.68427076e-06],
                      [1.44266191e-01, 8.72165215e-02, 7.50324113e-01, 1.81931744e-02]],
              
                     [[6.53246982e-01, 1.27680487e-01, 2.19068319e-01, 4.21227732e-06],
                      [3.02091515e-01, 5.08625854e-01, 1.75845651e-01, 1.34369793e-02],
                      [1.24160007e-01, 1.10927796e-01, 7.51893911e-01, 1.30182866e-02]],
              
                     [[6.04948021e-01, 8.56430966e-02, 2.93572002e-01, 1.58368801e-02],
                      [2.08021673e-01, 5.88787537e-01, 1.83461431e-01, 1.97293588e-02],
                      [7.40985207e-02, 1.08597875e-01, 8.02298872e-01, 1.50047320e-02]],
              
                     [[6.92428400e-01, 9.30498572e-02, 2.14512094e-01, 9.